## Fisrt try on tabular data and boosting model
- I didn't use much time to do this, so there is no emssemble & voting & tuning..

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
BASE_DIR = "../input/devkor-recruit-prediction"
train_data = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))
test_data = pd.read_csv(os.path.join(BASE_DIR, "test.csv"))

In [3]:
train_data.describe()

,ID,HW1,HW2,HW3,HW4,HW5,Interview,Pass
count,8000.00000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,3999.50000,5.042775,7.986415,3.607060,6.949719,4.998042,4.992632,0.172250
std,2309.54541,2.741636,0.972519,3.446848,1.887212,1.009120,1.989098,0.377621
min,0.00000,0.000000,4.340000,0.000000,0.000000,1.580000,0.000000,0.000000
25%,1999.75000,3.010000,7.300000,0.000000,5.660000,4.320000,3.640000,0.000000
50%,3999.50000,5.050000,8.000000,2.900000,7.010000,5.010000,4.990000,0.000000
75%,5999.25000,7.060000,8.660000,6.360000,8.370000,5.690000,6.350000,0.000000
max,7999.00000,10.000000,10.000000,10.000000,10.000000,8.750000,10.000000,1.000000


In [4]:
from sklearn.metrics import f1_score


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [6]:
target = train_data['Pass']
features = train_data.drop('ID', axis = 1, inplace = False).drop('Pass', axis=1, inplace = False)
#ID 제거, target 분리

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2,
     random_state = 34, stratify = target)
# 'Pass'의 평균값이 0.1로 1의 분포가 훨씬 적으므로 데이터셋을 나눌때 이의 분포도를 맞춰줄 필요가 있다.

In [7]:
dt_clf = DecisionTreeClassifier()
lgbm_clf = LGBMClassifier(n_estimators=500)
xgb_clf = XGBClassifier(eval_metric = 'logloss')

In [8]:
X_train.head()

,HW1,HW2,HW3,HW4,HW5,Interview
4614,1.89,8.41,0.00,4.37,5.08,5.11
2853,3.62,8.25,0.00,7.14,5.48,5.44
590,4.16,6.89,0.00,7.60,5.54,5.14
7897,7.98,6.78,2.01,7.06,5.13,4.76
1982,0.00,7.49,0.00,7.21,4.43,3.75


In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
skf = StratifiedKFold(n_splits = 10, shuffle=True, random_state=0)
dt_score = cross_val_score(dt_clf, X_train, y_train, scoring = 'f1', cv = skf)
lgbm_score = cross_val_score(lgbm_clf, X_train, y_train, scoring = 'f1', cv = skf)
xgb_score = cross_val_score(xgb_clf, X_train, y_train, scoring = 'f1', cv = skf)


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encode

In [10]:
print(np.mean(dt_score))
print(np.mean(lgbm_score))

0.8576696024865861
0.9426459313070497


In [11]:
sample_submission = pd.read_csv(os.path.join(BASE_DIR, "sample_submission.csv"))
sample_submission.head()

,ID,Pass
0,8000,0
1,8001,0
2,8002,0
3,8003,0
4,8004,0


In [12]:
lgbm_clf.fit(X_train, y_train)

LGBMClassifier(n_estimators=500)

In [13]:
y_pred = lgbm_clf.predict(test_data.drop('ID', axis=1))


In [14]:
sample_submission.loc[:, "Pass"] = y_pred
sample_submission["Pass"] = sample_submission["Pass"].astype(int)

In [15]:
sample_submission.describe()

,ID,Pass
count,2000.000000,2000.000000
mean,8999.500000,0.170500
std,577.494589,0.376166
min,8000.000000,0.000000
25%,8499.750000,0.000000
50%,8999.500000,0.000000
75%,9499.250000,0.000000
max,9999.000000,1.000000


In [16]:
sample_submission.to_csv("result.csv", index =False)